## minplascalc use case

Some thoughts on how people might use the minplascalc module for the most part in their workflow.

In [ ]:
import minplascalc as mpc

### Database interface

Return an dictionary- or list-like object containing representations of all species which are available in the user's local database, for the user to interact with in order to build mixtures and do calculations on them:

In [ ]:
mpcdata = mpc.database()

We should be able to show a string(?) list of keys representing species available in the mpcdb object, eg:

```
['e', 'N2', 'N2+', 'N', 'N+', 'N++', 'O2', 'O2+', 'O', 'O+', ...]
```

It would be nice if the list could also take a pattern for matching/filtering, e.g. "give me a sublist of all species containing oxygen", but that's a nice-to-have and not critical (and could be done by the user with list comprehension on the mpcdata object, I guess).

In [ ]:
mpcdata.list()
mpcdata.list(pattern_to_match)

If a species is not available in the local database but the user has data for it (or knows it exists on NIST), there should be methods for adding it into the JSON stash/relational DB/whatever disk data structure lies underneath minplascalc:

In [ ]:
mpcdata.add_monatomic_species(string_key, species, properties, here)
mpcdata.add_diatomic_species(string_key, species, properties, here)
mpcdata.add_monatomic_species_from_nist(string_key, web-scraping, arguments, here)

### Species objects

Instances of the `mpc.Species` object associated with a text key should be easily creatable from the database object, both explicitly and inside the minplascalc module's own code:

In [ ]:
my_sp = mpcdata['O2']

The Species object should probably expose nothing to the user except as read-only via a repr print method. Apart from testing or idle curiosity, it's hard to imagine any cases where end users would want to create a Species object in isolation, or access its various properties or methods for partition functions etc. These are always going to be called by higher-level calculations related to mixtures, not in their own right.

In [ ]:
my_sp. # Pressing tab should probably show nothing...
print(my_sp) # Show a nice formatted representation

### Plasma mixture calculations

This is really the reason that people will use minplascalc - to create mixtures of plasma species, and then do thermodynamic and thermophysical property calculations on those mixtures. The background machinery necessary to make this happen won't interest them overly much for the most part.

So what defines a "mixture"? It's basically a group of plasma species that can interact and react thermodynamically with each other to form an equilibrium composition, just like any normal chemical mixture. In order to calculate the equilibrium composition (and hence all the other thermodynamic and thermophysical properties of interest, which depend on the composition) we need to tell minplascalc which species are present, the temperature and pressure, and some mass balance information to constrain the minimisation problem; this latter is generally given as an initial gas composition at room temperature or some similar standard condition.

A typical minplascalc user can be expected to be competent enough to know how to select all the species which are important to their particular problem, and spec an appropriate initial composition by mole fraction. We'd generally want to then proceed directly to doing property calculations using that mixture spec. At the moment we do this by creating an intermediate `mpc.Mixture` object which holds the species present as well as setting up some useful variables and placeholders for the Gibbs free energy minimisation calc. The Mixture object is currently generated from a JSON file which as discussed is probably dumb for a number of reasons - it would be cleaner to just use a standard Python data structure like a dictionary or similar, as shown in the following cell.

Whether the Mixture class actually needs to be exposed explicitly at all is also open to debate - perhaps it would be more transparent for the user to simply call the property calculation functions directly with the initial composition and state variables as arguments, with the `mpc.Mixture` object created internally as needed?

In [ ]:
oxygen_initial_comp = {'O2': 1, 'O2+': 0, 'O': 0, 'O+': 0, 'O++': 0, 'e': 0}
oxygen_mix = mpc.Mixture(oxygen_initial_comp, T=10000, P=101325)

oxygen_mix. # Pressing tab should(?) show the methods available for equilibrium 
            # composition, density, enthalpy, heat capacity, and any other
            # calculations available.
            # It should also give access to properties such as the species in 
            # the mixtures (as Species objects), the temperature and pressure,
            # (other...?)

It's uncommon to want to see the equilibrium composition in isolation (it's usually a means to an end on the way to calculating a more useful property), but every now and then it's a good sanity check that plasma engineers might want to use - so that probably needs to be exposed directly in some way. In the example below, `eq_comp` would be a data structure similar to `oxygen_initial_comp` above... basically the user would appreciate having some way of getting at the actual equilibrium composition values calculated by the Gibbs free energy minimiser.

In [ ]:
oxygen_initial_comp = {'O2': 1, 'O2+': 0, 'O': 0, 'O+': 0, 'O++': 0, 'e': 0}
oxygen_mix = mpc.Mixture(oxygen_initial_comp, T=10000, P=101325)

eq_comp = oxygen_mix.equilibrium_composition()
# or as a module-level function...
eq_comp = mpc.equilibrium_composition(oxygen_mix)
# or as a module-level function with no explicit Mixture object...
eq_comp = mpc.equilibrium_composition(oxygen_initial_comp, T=10000, P=101325)

The mixture property calculators *definitely* need to be exposed directly, these are the main raison d'être for minplascalc. Again it's less common to perform a scalar calculation at a single temperature and pressure, but it's often a good sanity check (in addition to being useful for unit testing and accuracy comparisons etc):

In [ ]:
oxygen_initial_comp = {'O2': 1, 'O2+': 0, 'O': 0, 'O+': 0, 'O++': 0, 'e': 0}
oxygen_mix = mpc.Mixture(oxygen_initial_comp, T=10000, P=101325)

rho = oxygen_mix.density()
# or as a module-level function...
rho = mpc.density(oxygen_mix)
# or as a module-level function with no explicit Mixture object...
rho = mpc.density(oxygen_initial_comp, T=10000, P=101325)

Cp = oxygen_mix.heat_capacity()
# or as a module-level function...
Cp = mpc.heat_capacity(oxygen_mix)
# or as a module-level function with no explicit Mixture object...
Cp = mpc.heat_capacity(oxygen_initial_comp, T=10000, P=101325)

I foresee the main use case for miplascalc being running calculations of plasma properties in batch, across a range of state variable values. Output should probably be in standard Python data structures of some sort which the user can then take forward into their own code:

In [ ]:
import numpy as np
temperatures = np.linspace(2000, 25000, 100)
oxygen_initial_comp = {'O2': 1, 'O2+': 0, 'O': 0, 'O+': 0, 'O++': 0, 'e': 0}

oxygen_mixes = [mpc.Mixture(oxygen_initial_comp, T=tmp, P=101325) for tmp in temperatures]
rhos = [mym.density() for mym in oxygen_mixes]
# or as a module-level function...
oxygen_mixes = [mpc.Mixture(oxygen_initial_comp, T=tmp, P=101325) for tmp in temperatures]
rhos = [mpc.density(mym) for mym in oxygen_mixes]
# or as a module-level function with no explicit Mixture objects...
rhos = [mpc.density(oxygen_initial_comp, T=tmp, P=101325) for tmp in temperatures]

The user will typically then go away and do their own thing with the `temperatures` and `rhos` lists such as write out to files of various formats, draw graphs, and so on. Some utilities in minplascalc to assist with those tasks might be nice to have here, but I'm happy for us to assume that the user will know best what to do with their results from this point on.